In [1]:
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


## Libraries

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import string
import re
import wandb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers, models

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Constants

In [4]:
BATCH_SIZE = 32
MAX_FEATURES = 10000
SEQ_LENGTH = 250
EMBEDDING_DIM = 300
RNN_UNITS = 64

In [5]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac")

wandb: Currently logged in as: petar-boskovic-ac. Use `wandb login --relogin` to force relogin


Download the artifacts

In [6]:
train_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_train_transformed:v0', type='dataset')
test_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_test_transformed:v0', type='dataset')
val_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_val_transformed:v0', type='dataset')

artifact_dir = train_artifact.download()
artifact_dir = test_artifact.download()
artifact_dir = val_artifact.download()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [7]:
wandb.finish()

Transform into tfds

In [8]:
def df_to_tfds(dataframe, shuffle=False, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('Sentiment')
    ds = tf.data.Dataset.from_tensor_slices((df['Text'].values, labels.values))
    if shuffle==True:
        ds = ds.shuffle(buffer_size=len(dataframe))
    # ds = ds.batch(batch_size=batch_size)
    return ds

In [9]:
train_df = pd.read_csv('/content/artifacts/stock_data_train_transformed:v0/stock_data_train_transformed.csv')
val_df = pd.read_csv('/content/artifacts/stock_data_val_transformed:v0/stock_data_val_transformed.csv')
test_df = pd.read_csv('/content/artifacts/stock_data_test_transformed:v0/stock_data_test_transformed.csv')

In [10]:
train_ds = df_to_tfds(train_df)
val_ds = df_to_tfds(val_df)
test_ds = df_to_tfds(test_df)

In [11]:
for a,b in train_ds.take(1):
    print(a)
    print(b)

tf.Tensor(b'10 item le made two week shoestring budget writerdirector brad silberling little film shot carson ca feel like entire story improvisedin best sense word silberling good fortune pair veteran actor morgan freeman big project spanish actress paz vega result dialogue two people different vantage manage enhance life morgan freeman play yet part comedy depicted actor work four year scouting location little filler film get back flow thing role market manager dropped seedy market carson encounter among others one scarlet girl argumentative 10 item le checkout line scarlet tired static job also generally angry philandering husband bobby cannavale currently sleeping scarlet lazy coworker anne dudek lack ability get decent job elsewhere two pair shared problem go road trip result character growing presence life story simple story simply told tender bonding freeman paz work well one little film human relationship vulnerable change exchange message well worth viewing dvd featurettes tou

In [11]:
vectorize_layer = TextVectorization(max_tokens=MAX_FEATURES,
                                    output_mode='int',
                                    output_sequence_length=SEQ_LENGTH)

In [12]:
# Adapt the vectorize layer to the training text data
train_text = train_df['Text'].values
vectorize_layer.adapt(train_text)

In [13]:
len(vectorize_layer.get_vocabulary())

10000

In [14]:
vectorize_layer.get_vocabulary()[555]

'message'

In [13]:
def vectorize_text(text, label):
    #text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [14]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

In [16]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [17]:
for a,b in train_ds.take(1):
    print(a)
    print(b)

tf.Tensor(
[ 202 2928  222   28   40  781 7876  289 2463 2849    1   48    3  135
 6275 8358   94    5  332    9    1   47  195  257    1    7 3047 1650
 1845   38 2162 2805  106  802 1830  327 7991 2875  551  285   40   19
  189    1 1710 6493   31 2162 2805   72  153   56  102 2068   38   55
  559   46    1  757   48 4374    3   11   62 2007   27   78 1773 2665
 3081 6136 1773 6275 1482  650  281    4 7137   84    1  202 2928  222
    1  116 7137 1252 4742  179   21 1140 1493    1  482 2725    1 3410
 2638 7137 2707 8238 1661    1  338  878   11  437  179 2909   40 1650
 4848  194   32 1057 1074  551    8 1673 1129   31    9  501    9  232
  457 3890 8587 2805 7991   55   17    4   48    3  250  368 4257  359
 3271  555   17  185  609  171    1 1185 5458  582 1316  451 9981 9018
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [15]:
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

## Modeling

###Recurrent Neural Network (RNN)

In [29]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='Simple_RNN', config={'model_type': 'Simple RNN',
                                                                                                          'embedding_dim': 300,
                                                                                                          'rnn_units': 64,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [35]:
simple_rnn = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # SimpleRNN with BatchNormalization and Dropout
                                  tf.keras.layers.SimpleRNN(units=RNN_UNITS, return_sequences=False),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

simple_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

simple_rnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 64)                  │          23,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,028,097 (11.55 MB)

 Trainable params: 3,027,841 (11.55 MB)

 Non-trainable params: 256 (1.00 KB)

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
print("Wandb version:", wandb.__version__)
print("TensorFlow version:", tf.__version__)

Wandb version: 0.17.7
TensorFlow version: 2.17.0


In [17]:
wandb_logger = wandb.keras.WandbMetricsLogger()

Error: You must call `wandb.init()` before WandbMetricsLogger()

In [37]:
history = simple_rnn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 223s 170ms/step - AUC: 0.4974 - accuracy: 0.5024 - loss: 0.7728 - val_AUC: 0.5000 - val_accuracy: 0.5012 - val_loss: 0.6953
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 233s 179ms/step - AUC: 0.5125 - accuracy: 0.5103 - loss: 0.7002 - val_AUC: 0.4937 - val_accuracy: 0.4992 - val_loss: 0.6946
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 214s 165ms/step - AUC: 0.5052 - accuracy: 0.5108 - loss: 0.6954 - val_AUC: 0.5056 - val_accuracy: 0.5077 - val_loss: 0.6941
Epoch 4/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 212s 163ms/step - AUC: 0.5134 - accuracy: 0.5106 - loss: 0.6934 - val_AUC: 0.5185 - val_accuracy: 0.5179 - val_loss: 0.6938
Epoch 5/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 197s 152ms/step - AUC: 0.5260 - accuracy: 0.5224 - loss: 0.6926 - val_AUC: 0.5181 - val_accuracy: 0.5179 - val_loss: 0.6945
Epoch 6/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 199s 153ms/step - AUC: 0.5265 - accuracy: 0.5201 - loss: 0.6926 - val_AUC: 0.5186 - val_accuracy: 0.5173 - val_loss: 0.6944
Epoc

In [44]:
simple_rnn.save('simple_rnn.keras')

In [45]:
artifact = wandb.Artifact('simple_rnn',
                          type='model',
                          description='Simple RNN model for sentiment analysis',
                          metadata={'epochs': wandb.config.epochs,
                                    'learning_rate': 1e-3,
                                    'batch_size': wandb.config.as_dict,
                                    'architecture': 'Simple RNN'})

artifact.add_file('/content/simple_rnn.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▂▅▇█▁▂▁▂▃▃▃
epoch/accuracy,▁▂▁▃█▇█
epoch/epoch,▁▂▃▅▆▁▂▃▅▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,▇▄▃▂▁█▅▄▄▄▄▄
epoch/val_AUC,▄▂▁▃▂▃▁▅███▇
epoch/val_accuracy,▂▁▄███▇
epoch/val_loss,▁▂▅█▂▁▁▁▁▁▁▁
epoch/AUC,0.52713
epoch/accuracy,0.52174
epoch/epoch,6


####Evaluation

In [46]:
test_ds = test_ds.batch(BATCH_SIZE)
simple_rnn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - AUC: 0.5419 - accuracy: 0.5446 - loss: 0.6905


[0.6933395266532898, 0.5222451686859131, 0.5293551683425903]

###LSTM RNN

In [48]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='RNN_LSTM', config={'model_type': 'RNN LSTM',
                                                                                                          'embedding_dim': 300,
                                                                                                          'rnn_units': 64,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [49]:
lstm = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # LSTM with BatchNormalization and Dropout
                                  tf.keras.layers.LSTM(units=RNN_UNITS, return_sequences=False),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

lstm.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,098,177 (11.82 MB)

 Trainable params: 3,097,921 (11.82 MB)

 Non-trainable params: 256 (1.00 KB)

In [50]:
history = lstm.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 518s 395ms/step - AUC: 0.5074 - accuracy: 0.5034 - loss: 0.7453 - val_AUC: 0.5932 - val_accuracy: 0.5897 - val_loss: 0.6702
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 552s 387ms/step - AUC: 0.6191 - accuracy: 0.5941 - loss: 0.6709 - val_AUC: 0.6357 - val_accuracy: 0.6257 - val_loss: 0.6588
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 501s 386ms/step - AUC: 0.6139 - accuracy: 0.5802 - loss: 0.6660 - val_AUC: 0.5540 - val_accuracy: 0.5121 - val_loss: 0.9002
Epoch 4/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 522s 402ms/step - AUC: 0.6156 - accuracy: 0.5673 - loss: 0.6525 - val_AUC: 0.9278 - val_accuracy: 0.8543 - val_loss: 0.3536
Epoch 5/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 509s 392ms/step - AUC: 0.9421 - accuracy: 0.8711 - loss: 0.3067 - val_AUC: 0.9448 - val_accuracy: 0.8662 - val_loss: 0.3181
Epoch 6/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 503s 387ms/step - AUC: 0.9730 - accuracy: 0.9220 - loss: 0.2058 - val_AUC: 0.9332 - val_accuracy: 0.8570 - val_loss: 0.4392
Epoc

In [51]:
lstm.save('rnn_lstm.keras')

In [53]:
artifact = wandb.Artifact('rnn_lstm',
                          type='model',
                          description='RNN model with LSTM for sentiment analysis',
                          metadata={'epochs': 15,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'RNN LSTM'})

artifact.add_file('/content/rnn_lstm.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▃▃▄▇███
epoch/accuracy,▁▂▂▃▇▇██
epoch/epoch,▁▂▃▄▅▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▆▃▂▁▁
epoch/val_AUC,▂▂▁█████
epoch/val_accuracy,▂▃▁█████
epoch/val_loss,▅▅█▁▁▂▂▃
epoch/AUC,0.99254
epoch/accuracy,0.97052
epoch/epoch,7


####Evaluation

In [54]:
lstm.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 17s 104ms/step - AUC: 0.9467 - accuracy: 0.8717 - loss: 0.3140


[0.3224245309829712, 0.9439403414726257, 0.8671799898147583]

###Bidirectional LSTM RNN

In [55]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='RNN_LSTM_Bidirectional', config={'model_type': 'RNN LSTM Bidirectional',
                                                                                                          'embedding_dim': 300,
                                                                                                          'rnn_units': 64,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [57]:
lstm_binn = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # LSTM with Bidirectional wrapper, BatchNormalization and Dropout
                                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=RNN_UNITS, return_sequences=False)),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

lstm_binn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

lstm_binn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,195,969 (12.19 MB)

 Trainable params: 3,195,585 (12.19 MB)

 Non-trainable params: 384 (1.50 KB)

In [58]:
history = lstm_binn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 989s 755ms/step - AUC: 0.8104 - accuracy: 0.7355 - loss: 0.5311 - val_AUC: 0.9380 - val_accuracy: 0.8695 - val_loss: 0.3338
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 964s 742ms/step - AUC: 0.9522 - accuracy: 0.8868 - loss: 0.2783 - val_AUC: 0.9412 - val_accuracy: 0.8435 - val_loss: 0.4273
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 955s 735ms/step - AUC: 0.9738 - accuracy: 0.9217 - loss: 0.2037 - val_AUC: 0.9335 - val_accuracy: 0.8556 - val_loss: 0.4145


In [59]:
lstm_binn.save('rnn_lstm_bidirectional.keras')

In [60]:
artifact = wandb.Artifact('rnn_lstm_bidirectional',
                          type='model',
                          description='Bidirectional RNN model with LSTM for sentiment analysis',
                          metadata={'epochs': 15,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'Bidirectional RNN LSTM'})

artifact.add_file('/content/rnn_lstm_bidirectional.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▆█
epoch/accuracy,▁▆█
epoch/epoch,▁▅█
epoch/learning_rate,▁▁▁
epoch/loss,█▃▁
epoch/val_AUC,▅█▁
epoch/val_accuracy,█▁▄
epoch/val_loss,▁█▇
epoch/AUC,0.97749
epoch/accuracy,0.92831
epoch/epoch,2


####Evaluation

In [61]:
lstm_binn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 32s 193ms/step - AUC: 0.9427 - accuracy: 0.8701 - loss: 0.3246


[0.33423614501953125, 0.9379280209541321, 0.8631376028060913]

###Bidirectional GRU RNN

In [68]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='RNN_GRU_Bidirectional', config={'model_type': 'RNN GRU Bidirectional',
                                                                                                          'embedding_dim': 300,
                                                                                                          'rnn_units': 64,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [63]:
gru_binn = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # LSTM with Bidirectional wrapper, BatchNormalization and Dropout
                                  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=RNN_UNITS, return_sequences=False)),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

gru_binn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

gru_binn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │         140,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,149,633 (12.01 MB)

 Trainable params: 3,149,249 (12.01 MB)

 Non-trainable params: 384 (1.50 KB)

In [64]:
history = gru_binn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 995s 761ms/step - AUC: 0.7607 - accuracy: 0.6954 - loss: 0.5891 - val_AUC: 0.9361 - val_accuracy: 0.5799 - val_loss: 1.0849
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 948s 730ms/step - AUC: 0.9566 - accuracy: 0.8900 - loss: 0.2655 - val_AUC: 0.9400 - val_accuracy: 0.8587 - val_loss: 0.3449
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 940s 724ms/step - AUC: 0.9803 - accuracy: 0.9327 - loss: 0.1758 - val_AUC: 0.9287 - val_accuracy: 0.8404 - val_loss: 0.5067


In [65]:
gru_binn.save('rnn_gru_bidirectional.keras')

In [69]:
artifact = wandb.Artifact('rnn_gru_bidirectional',
                          type='model',
                          description='Bidirectional RNN model with GRU for sentiment analysis',
                          metadata={'epochs': 15,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'Bidirectional RNN GRU'})

artifact.add_file('/content/rnn_gru_bidirectional.keras')
wandb.log_artifact(artifact)

wandb.finish()

####Evaluation

In [70]:
gru_binn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 28s 172ms/step - AUC: 0.9390 - accuracy: 0.5842 - loss: 1.0613


[1.0777918100357056, 0.9364612102508545, 0.583060622215271]

###Bidirectional Multilayered GRU RNN

In [18]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='RNN_GRU_Bidirectional_Multilayered',
                                                                                                        config={'model_type': 'RNN GRU Bidirectional Multilayered',
                                                                                                        'embedding_dim': 300,
                                                                                                        'initial_rnn_units': 64,
                                                                                                        'batch_size': 32,
                                                                                                        'epochs': 15})

In [23]:
multi_gru_binn = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # GRU with Bidirectional wrapper
                                  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=RNN_UNITS, return_sequences=True)),

                                  # GRU with Bidirectional wrapper, BatchNormalization and Dropout
                                  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=RNN_UNITS//2, return_sequences=False)),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

multi_gru_binn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

multi_gru_binn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 250, 128)            │         140,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          31,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,176,385 (12.12 MB)

 Trainable params: 3,176,129 (12.12 MB)

 Non-trainable params: 256 (1.00 KB)

In [24]:
history = multi_gru_binn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1366s 1s/step - AUC: 0.7764 - accuracy: 0.7060 - loss: 0.5651 - val_AUC: 0.9552 - val_accuracy: 0.8806 - val_loss: 0.2824
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1347s 1s/step - AUC: 0.9607 - accuracy: 0.8972 - loss: 0.2524 - val_AUC: 0.9467 - val_accuracy: 0.8712 - val_loss: 0.3180
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1346s 994ms/step - AUC: 0.9810 - accuracy: 0.9371 - loss: 0.1716 - val_AUC: 0.9405 - val_accuracy: 0.8693 - val_loss: 0.3774
Epoch 4/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1283s 987ms/step - AUC: 0.9920 - accuracy: 0.9621 - loss: 0.1058 - val_AUC: 0.9414 - val_accuracy: 0.8708 - val_loss: 0.3911


In [25]:
multi_gru_binn.save('rnn_gru_bidirectional_multi.keras')

In [26]:
artifact = wandb.Artifact('rnn_gru_bidirectional_multi',
                          type='model',
                          description='Bidirectional Multilayered RNN model with GRU for sentiment analysis',
                          metadata={'epochs': 15,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'Bidirectional Multilayered RNN GRU'})

artifact.add_file('/content/rnn_gru_bidirectional_multi.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▆██
epoch/accuracy,▁▆▇█
epoch/epoch,▁▃▆█
epoch/learning_rate,▁▁▁▁
epoch/loss,█▄▂▁
epoch/val_AUC,█▄▁▁
epoch/val_accuracy,█▂▁▂
epoch/val_loss,▁▃▇█
epoch/AUC,0.99322
epoch/accuracy,0.96559
epoch/epoch,3


####Evaluation

In [23]:
multi_gru_binn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 41s 240ms/step - AUC: 0.9592 - accuracy: 0.8857 - loss: 0.2720


[0.2862298786640167, 0.9541996717453003, 0.8806544542312622]